In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../country-and-continent-codes-list-csv_csv.csv")

In [3]:
df

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0
...,...,...,...,...,...,...
257,Africa,AF,"Zambia, Republic of",ZM,ZMB,894.0
258,Oceania,OC,Disputed Territory,XX,NaN,NaN
259,Asia,AS,Iraq-Saudi Arabia Neutral Zone,XE,NaN,NaN
260,Asia,AS,United Nations Neutral Zone,XD,NaN,NaN


In [ ]:
import csv 

def get_source_counts(targetCountry, db, converter):
    shortCountryCode = converter.fipsNameToCode(targetCountry)
    query_str = '''
    SELECT 
        MentionSourceName, count(*) as sourceMentionCount 
    FROM 
        eventmentions 
    WHERE 
        GLOBALEVENTID in (SELECT 
                              GLOBALEVENTID 
                          FROM 
                              events 
                          WHERE 
                              (Actor1CountryCode = "CHN" OR Actor2CountryCode = "CHN") AND 
                              ActionGeo_CountryCode = "''' + shortCountryCode + '''"
                         ) 
    GROUP BY 
        MentionSourceName 
    ORDER BY 
        sourceMentionCount DESC;
    '''
    
    fields = ['Source Name', 'Mention Count']
    with open(targetCountry + "_sourceCounts.csv", 'w') as csvfile:
        csvwriter = csv.writer(csvfile) 
        csvwriter.writerow(fields) 
        
        cursor = db.cursor()
        cursor.execute(query_str)
        result = cursor.fetchall()
        cursor.close()
        print(result)

        for source_counts in result:
            csvwriter.writerow([source_counts[0], source_counts[1]])


In [ ]:
targetCountry = 'Afghanistan'

In [ ]:
import mysql.connector

db = mysql.connector.connect(
    host="gdelt-events-mentions.cbghl9qobm13.us-east-2.rds.amazonaws.com",
    user="admin",
    passwd="R1m23&!#Bto",
    database="gdelt")

In [4]:
from Python.countryCodeConversion import countryCodeConversion

converter = countryCodeConversion()

In [ ]:
get_source_counts(targetCountry, db, converter)

In [ ]:
db.close()

In [5]:
df_asia = df.loc[df.loc[:,"Continent_Name"] == "Asia", :]

In [6]:
df_asia_sutter = pd.concat([df_asia.loc[:,["Country_Name", "Two_Letter_Country_Code"]], 
                            df_asia.loc[:,"Country_Name"].apply(lambda x: converter.fipsNameToCode(x.split(',')[0]))], axis=1)

In [7]:
df_asia_sutter.columns = ["Country_Name", "Two_Letter_Country_Code", "GDELT Code"]

In [8]:
df_asia_sutter

,Country_Name,Two_Letter_Country_Code,GDELT Code
0,"Afghanistan, Islamic Republic of",AF,AF
9,"Azerbaijan, Republic of",AZ,AJ
14,"Bahrain, Kingdom of",BH,BA
15,"Bangladesh, People's Republic of",BD,BG
17,"Armenia, Republic of",AM,AM
21,"Bhutan, Kingdom of",BT,BT
28,British Indian Ocean Territory (Chagos Archipe...,IO,1
31,Brunei Darussalam,BN,BX
33,"Myanmar, Union of",MM,BM
36,"Cambodia, Kingdom of",KH,CB


In [9]:
'''
Country = FIPS Code

British Indian Ocean Territory = IO
Palestinian Territory, Occupied = WE
Korea, Democratic People's Republic of = KN
Korea, Republic of = KS
Kyrgyz Republic = KG
Lao People's Democratic Republic = LA
Vietnam, Socialist Republic of = VM


Iraq-Saudi Arabia Neutral Zone, United Nations Neutral Zone, Spratly Islands are not included.
'''

"\nCountry = FIPS Code\n\nBritish Indian Ocean Territory = IO\nPalestinian Territory, Occupied = WE\nKorea, Democratic People's Republic of = KN\nKorea, Republic of = KS\nKyrgyz Republic = KG\nLao People's Democratic Republic = LA\nVietnam, Socialist Republic of = VM\n\n\nIraq-Saudi Arabia Neutral Zone, United Nations Neutral Zone, Spratly Islands are not included.\n"

In [10]:
df_asia_sutter.loc[df_asia_sutter.loc[:,"Two_Letter_Country_Code"] == "IO", "GDELT Code"]= "IO"
df_asia_sutter.loc[df_asia_sutter.loc[:,"Two_Letter_Country_Code"] == "PS", "GDELT Code"]= "WE"
df_asia_sutter.loc[df_asia_sutter.loc[:,"Two_Letter_Country_Code"] == "KP", "GDELT Code"]= "KN"
df_asia_sutter.loc[df_asia_sutter.loc[:,"Two_Letter_Country_Code"] == "KR", "GDELT Code"]= "KS"
df_asia_sutter.loc[df_asia_sutter.loc[:,"Two_Letter_Country_Code"] == "KG", "GDELT Code"]= "KG"
df_asia_sutter.loc[df_asia_sutter.loc[:,"Two_Letter_Country_Code"] == "LA", "GDELT Code"]= "LA"
df_asia_sutter.loc[df_asia_sutter.loc[:,"Two_Letter_Country_Code"] == "VN", "GDELT Code"]= "VM"

In [11]:
df_asia_sutter = df_asia_sutter.iloc[:-3,:]

In [ ]:
import csv
import os
from google.cloud import bigquery

def get_source_counts_bq(target_country, target_fips_code, bq_client):
    bq_query_str = '''
    SELECT 
        MentionSourceName, count(*) as sourceMentionCount 
    FROM 
        gdelt-bq.gdeltv2.eventmentions_partitioned m 
    WHERE 
        m.GLOBALEVENTID in (SELECT 
                                  GLOBALEVENTID 
                            FROM 
                                  gdelt-bq.gdeltv2.events_partitioned e 
                            WHERE 
                                  (e.Actor1CountryCode = "CHN" OR e.Actor2CountryCode = "CHN") AND 
                                  e.ActionGeo_CountryCode = "''' + target_fips_code  + '''" AND
                                  e._PARTITIONTIME BETWEEN TIMESTAMP('2015-02-01') AND TIMESTAMP('2019-05-31')) AND
        m._PARTITIONTIME BETWEEN TIMESTAMP('2015-02-01') AND TIMESTAMP('2019-05-31')
    GROUP BY 
        MentionSourceName 
    ORDER BY 
        sourceMentionCount DESC;
    '''
    
    job_config = bigquery.QueryJobConfig()
    job_config.dry_run = True  
    query_job = bq_client.query(bq_query_str, job_config)
    gb_to_be_processed = query_job.total_bytes_processed/1024/1024/1024
    print("\n\nCountry: {}\nBytes (GB) to be processed: {}".format(target_country, gb_to_be_processed))

    if gb_to_be_processed > 100:
        print("Exiting! BQ bytes to be processed too large")
        return
        
    fields = ['Source Name', 'Mention Count']
    target_country_dir = 'CountrySourceCounts/' + target_country
    os.mkdir(target_country_dir)
    with open(target_country_dir + "/sourceCounts.csv", 'w') as csvfile:
        csvwriter = csv.writer(csvfile) 
        csvwriter.writerow(fields) 
        
        query_job = bq_client.query(bq_query_str)
        
        for row in query_job:
            csvwriter.writerow([row[0], row[1]])
        
        print("Bytes (GB) actually processed: {}".format(query_job.total_bytes_processed/1024/1024/1024))

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

for index, row in df_asia_sutter.iterrows():
    get_source_counts_bq(row['Country_Name'], row['GDELT Code'], client)

In [12]:
df_africa = df.loc[df.loc[:,"Continent_Name"] == "Africa", :]

In [13]:
df_africa_sutter = pd.concat([df_africa.loc[:,["Country_Name", "Two_Letter_Country_Code"]], 
                              df_africa.loc[:,"Country_Name"].
                              apply(lambda x: converter.fipsNameToCode(x.split(',')[0]))], axis=1)

In [14]:
df_africa_sutter.columns = ["Country_Name", "Two_Letter_Country_Code", "GDELT Code"]

In [15]:
df_africa_sutter

,Country_Name,Two_Letter_Country_Code,GDELT Code
3,"Algeria, People's Democratic Republic of",DZ,AG
6,"Angola, Republic of",AO,AO
24,"Botswana, Republic of",BW,BC
34,"Burundi, Republic of",BI,BY
37,"Cameroon, Republic of",CM,CM
39,"Cape Verde, Republic of",CV,1
41,Central African Republic,CF,CT
43,"Chad, Republic of",TD,CD
50,"Comoros, Union of the",KM,CN
51,Mayotte,YT,MF


In [16]:
'''
Country = FIPS Code

Cape Verde, Republic of (CV) = CV
Cote d'Ivoire, Republic of (CI) = IV
Libyan Arab Jamahiriya (LY) = LY
Saint Helena (SH) = SH
Tanzania, United Republic of (TZ) = TZ
Congo, Democratic Republic of the (CD) = CG
'''

"\nCountry = FIPS Code\n\nCape Verde, Republic of (CV) = CV\nCote d'Ivoire, Republic of (CI) = IV\nLibyan Arab Jamahiriya (LY) = LY\nSaint Helena (SH) = SH\nTanzania, United Republic of (TZ) = TZ\nCongo, Democratic Republic of the (CD) = CG\n"

In [17]:
df_africa_sutter.loc[df_africa_sutter.loc[:,"Two_Letter_Country_Code"] == "CV", "GDELT Code"]= "CV"
df_africa_sutter.loc[df_africa_sutter.loc[:,"Two_Letter_Country_Code"] == "CI", "GDELT Code"]= "IV"
df_africa_sutter.loc[df_africa_sutter.loc[:,"Two_Letter_Country_Code"] == "LY", "GDELT Code"]= "LY"
df_africa_sutter.loc[df_africa_sutter.loc[:,"Two_Letter_Country_Code"] == "SH", "GDELT Code"]= "SH"
df_africa_sutter.loc[df_africa_sutter.loc[:,"Two_Letter_Country_Code"] == "TZ", "GDELT Code"]= "TZ"
df_africa_sutter.loc[df_africa_sutter.loc[:,"Two_Letter_Country_Code"] == "CD", "GDELT Code"]= "CG"

In [18]:
df_africa_sutter

,Country_Name,Two_Letter_Country_Code,GDELT Code
3,"Algeria, People's Democratic Republic of",DZ,AG
6,"Angola, Republic of",AO,AO
24,"Botswana, Republic of",BW,BC
34,"Burundi, Republic of",BI,BY
37,"Cameroon, Republic of",CM,CM
39,"Cape Verde, Republic of",CV,CV
41,Central African Republic,CF,CT
43,"Chad, Republic of",TD,CD
50,"Comoros, Union of the",KM,CN
51,Mayotte,YT,MF


In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

for index, row in df_africa_sutter.iterrows():
    get_source_counts_bq(row['Country_Name'], row['GDELT Code'], client)

## Summary Stats

For each country, compute the mean tone of the event mentions from the **top** (local.csv) sources. 

In [19]:
import os

In [20]:
os.getcwd()

'/Users/None/nps/git_projects/csumb-capstone-fall-ay-2020/Sutter Laird'

In [21]:
os.listdir()

['ChinaActorSourceCountsInAsiaAfrica.csv',
 'LAIRDworkjournal.md',
 'Python',
 'Country Tables',
 'counts.csv',
 'CountrySourceCounts',
 'South Africa GKG',
 'logodraft1.png',
 'countsCompare.csv',
 'topwest_china.csv',
 'logodraft2.png',
 '.ipynb_checkpoints',
 'source_counts.ipynb',
 'suttercapstone.code-workspace',
 'Saved Documentation']

In [22]:
country_dirs = os.listdir('CountrySourceCounts')

In [23]:
country_dirs

['Israel, State of',
 'Somalia, Somali Republic',
 'Iraq, Republic of',
 'Tajikistan, Republic of',
 'Jordan, Hashemite Kingdom of',
 'Zimbabwe, Republic of',
 'Sierra Leone, Republic of',
 'Uzbekistan, Republic of',
 'Seychelles, Republic of',
 'Bahrain, Kingdom of',
 'Angola, Republic of',
 'Korea, Republic of',
 'Nepal, State of',
 'Mali, Republic of',
 'Guinea-Bissau, Republic of',
 'Burundi, Republic of',
 'Yemen',
 'Sri Lanka, Democratic Socialist Republic of',
 'Singapore, Republic of',
 'Qatar, State of',
 'Cape Verde, Republic of',
 'Togo, Togolese Republic',
 'Macao, Special Administrative Region of China',
 'South Sudan',
 'Georgia',
 'Ethiopia, Federal Democratic Republic of',
 'Equatorial Guinea, Republic of',
 'Morocco, Kingdom of',
 'Cocos (Keeling) Islands',
 "Algeria, People's Democratic Republic of",
 'Guinea, Republic of',
 'Zambia, Republic of',
 'Azerbaijan, Republic of',
 'Malaysia',
 "Bangladesh, People's Republic of",
 'Tunisia, Tunisian Republic',
 'British Ind

In [24]:
top_sources_filename = 'local.csv'

In [25]:
top_sources_paths = []
for d in country_dirs: 
    country_dir = os.path.join('CountrySourceCounts', d)
    files = os.listdir(country_dir)
    if top_sources_filename in files:
        top_sources_paths.append(os.path.join(country_dir, top_sources_filename))

In [26]:
top_sources_paths

['CountrySourceCounts/Israel, State of/local.csv',
 'CountrySourceCounts/Somalia, Somali Republic/local.csv',
 'CountrySourceCounts/Iraq, Republic of/local.csv',
 'CountrySourceCounts/Tajikistan, Republic of/local.csv',
 'CountrySourceCounts/Jordan, Hashemite Kingdom of/local.csv',
 'CountrySourceCounts/Zimbabwe, Republic of/local.csv',
 'CountrySourceCounts/Sierra Leone, Republic of/local.csv',
 'CountrySourceCounts/Uzbekistan, Republic of/local.csv',
 'CountrySourceCounts/Seychelles, Republic of/local.csv',
 'CountrySourceCounts/Bahrain, Kingdom of/local.csv',
 'CountrySourceCounts/Angola, Republic of/local.csv',
 'CountrySourceCounts/Korea, Republic of/local.csv',
 'CountrySourceCounts/Nepal, State of/local.csv',
 'CountrySourceCounts/Mali, Republic of/local.csv',
 'CountrySourceCounts/Guinea-Bissau, Republic of/local.csv',
 'CountrySourceCounts/Burundi, Republic of/local.csv',
 'CountrySourceCounts/Yemen/local.csv',
 'CountrySourceCounts/Sri Lanka, Democratic Socialist Republic of/

In [27]:
import pandas as pd

In [28]:
top_sources = {}
for path in top_sources_paths:
    df_sources = pd.read_csv(path)
    country = path.split('/')[1]
    top_sources[country] = list(df_sources['Source Name'])

In [29]:
top_sources

{'Israel, State of': ['timesofisrael.com',
  'jpost.com',
  'globes.co.il',
  'israelherald.com',
  'ynetnews.com',
  'algemeiner.com',
  'themarker.com',
  'haaretz.com',
  'israelnationalnews.com'],
 'Somalia, Somali Republic': ['hillaac.net',
  'radiomuqdisho.net',
  'somalilandsun.com',
  'qaranimo.com',
  'radioshabelle.com',
  'shabellenews.com',
  'midnimo.com',
  'garoweonline.com'],
 'Iraq, Republic of': ['iraqakhbar.com',
  'alsabaah.iq',
  'almadapaper.net',
  'akhbaar.org',
  'newsabah.com',
  'sotaliraq.com'],
 'Tajikistan, Republic of': ['news.tj',
  'khovar.tj',
  'avesta.tj',
  'asiaplus.tj'],
 'Jordan, Hashemite Kingdom of': ['petra.gov.jo',
  'jordantimes.com',
  'ammonnews.net',
  'alrai.com',
  'alghad.com',
  'alanbatnews.net',
  'albawaba.com',
  'sarayanews.com',
  'addustour.com',
  'khaberni.com'],
 'Zimbabwe, Republic of': ['bulawayo24.com',
  'herald.co.zw',
  'thezimbabwemail.com',
  'thezimbabwedaily.com',
  'chronicle.co.zw',
  'newsday.co.zw',
  'theindep

In [30]:
df_asia_sutter.head()

,Country_Name,Two_Letter_Country_Code,GDELT Code
0,"Afghanistan, Islamic Republic of",AF,AF
9,"Azerbaijan, Republic of",AZ,AJ
14,"Bahrain, Kingdom of",BH,BA
15,"Bangladesh, People's Republic of",BD,BG
17,"Armenia, Republic of",AM,AM


In [31]:
df_africa_sutter.head()

,Country_Name,Two_Letter_Country_Code,GDELT Code
3,"Algeria, People's Democratic Republic of",DZ,AG
6,"Angola, Republic of",AO,AO
24,"Botswana, Republic of",BW,BC
34,"Burundi, Republic of",BI,BY
37,"Cameroon, Republic of",CM,CM


In [32]:
country_name_gdelt_code = {}

In [33]:
for name, code in zip(df_asia_sutter.loc[:, 'Country_Name'], df_asia_sutter.loc[:, 'GDELT Code']):
    country_name_gdelt_code[name] = code

In [34]:
for name, code in zip(df_africa_sutter.loc[:, 'Country_Name'], df_africa_sutter.loc[:, 'GDELT Code']):
    country_name_gdelt_code[name] = code

In [35]:
country_name_gdelt_code

{'Afghanistan, Islamic Republic of': 'AF',
 'Azerbaijan, Republic of': 'AJ',
 'Bahrain, Kingdom of': 'BA',
 "Bangladesh, People's Republic of": 'BG',
 'Armenia, Republic of': 'AM',
 'Bhutan, Kingdom of': 'BT',
 'British Indian Ocean Territory (Chagos Archipelago)': 'IO',
 'Brunei Darussalam': 'BX',
 'Myanmar, Union of': 'BM',
 'Cambodia, Kingdom of': 'CB',
 'Sri Lanka, Democratic Socialist Republic of': 'CE',
 "China, People's Republic of": 'CH',
 'Taiwan': 'TW',
 'Christmas Island': 'KT',
 'Cocos (Keeling) Islands': 'CK',
 'Cyprus, Republic of': 'CY',
 'Georgia': 'GG',
 'Palestinian Territory, Occupied': 'WE',
 'Hong Kong, Special Administrative Region of China': 'HK',
 'India, Republic of': 'IN',
 'Indonesia, Republic of': 'ID',
 'Iran, Islamic Republic of': 'IR',
 'Iraq, Republic of': 'IZ',
 'Israel, State of': 'IS',
 'Japan': 'JA',
 'Kazakhstan, Republic of': 'KZ',
 'Jordan, Hashemite Kingdom of': 'JO',
 "Korea, Democratic People's Republic of": 'KN',
 'Korea, Republic of': 'KS',
 

In [36]:
top_sources_fips = {}
for k, v in top_sources.items():
    top_sources_fips[country_name_gdelt_code[k]] = v

In [37]:
top_sources_fips

{'IS': ['timesofisrael.com',
  'jpost.com',
  'globes.co.il',
  'israelherald.com',
  'ynetnews.com',
  'algemeiner.com',
  'themarker.com',
  'haaretz.com',
  'israelnationalnews.com'],
 'SO': ['hillaac.net',
  'radiomuqdisho.net',
  'somalilandsun.com',
  'qaranimo.com',
  'radioshabelle.com',
  'shabellenews.com',
  'midnimo.com',
  'garoweonline.com'],
 'IZ': ['iraqakhbar.com',
  'alsabaah.iq',
  'almadapaper.net',
  'akhbaar.org',
  'newsabah.com',
  'sotaliraq.com'],
 'TI': ['news.tj', 'khovar.tj', 'avesta.tj', 'asiaplus.tj'],
 'JO': ['petra.gov.jo',
  'jordantimes.com',
  'ammonnews.net',
  'alrai.com',
  'alghad.com',
  'alanbatnews.net',
  'albawaba.com',
  'sarayanews.com',
  'addustour.com',
  'khaberni.com'],
 'ZI': ['bulawayo24.com',
  'herald.co.zw',
  'thezimbabwemail.com',
  'thezimbabwedaily.com',
  'chronicle.co.zw',
  'newsday.co.zw',
  'theindependent.co.zw',
  'dailynews.co.zw',
  'sundaymail.co.zw',
  'newzimbabwe.com',
  'zimeye.net',
  'zbc.co.zw',
  'zimbabwest

**Local reporting** (i.e. popular local media) tone statistics for various actors (CHN, RUS, IND, USA, FRA, ALL) in the target country for the target time period.

In [ ]:
from google.cloud import bigquery

def get_summary_stats_bq(target_country, target_fips_code, top_sources, bq_client):
    sources_str = '("' + '", "'.join(top_sources) + '")'
    
    for actor in ['CHN', 'RUS', 'IND', 'USA', 'FRA', 'ALL']: # CAMEO Country Code
        actor_where = '(e.Actor1CountryCode = "' + actor + '" OR e.Actor2CountryCode = "' + actor + '") AND' if actor != 'ALL' else ''
        for start, end in [("2015-02-18", "2015-12-31"),
                           ("2016-01-01", "2016-12-31"), 
                           ("2017-01-01", "2017-12-31"), 
                           ("2018-01-01", "2018-12-31"), 
                           ("2019-01-01", "2019-12-31"), 
                           ("2020-01-01", "2020-12-31"),
                           ("2021-01-01", "2021-03-08"),
                           ("2015-02-18", "2021-03-08")]:
            bq_query_str = '''
            SELECT 
                avg(MentionDocTone) as AverageTone, stddev(MentionDocTone) as StdDevTone, count(MentionDocTone) as NumArticles
            FROM 
                gdelt-bq.gdeltv2.eventmentions_partitioned m 
            WHERE 
                m.GLOBALEVENTID in (SELECT 
                                          GLOBALEVENTID 
                                    FROM 
                                          gdelt-bq.gdeltv2.events_partitioned e 
                                    WHERE 
                                          ''' + actor_where + '''
                                          e.ActionGeo_CountryCode = "''' + target_fips_code + '''" AND
                                          e._PARTITIONTIME BETWEEN TIMESTAMP("''' + start + '''") AND 
                                          TIMESTAMP("''' + end + '''")) AND
                m._PARTITIONTIME BETWEEN TIMESTAMP("''' + start + '''") AND TIMESTAMP("''' + end + '''") AND
                m.MentionSourceName in ''' + sources_str + ''';
            '''

            print("\n\nCountry: {}\nActor: {}\nStart: {}\nEnd: {}".format(target_country, actor, start, end))
            fields = ['CountryCAMEOCode', 'StartYear', 'EndYear', 'Mean', 'StdDev', 'NumArticles']
            target_country_dir = 'CountrySourceCounts/' + target_country
            with open(target_country_dir + "/local_stats_" + start + "_" + end + "_" + actor + ".csv", 'w') as csvfile:
                csvwriter = csv.writer(csvfile) 
                csvwriter.writerow(fields) 

                query_job = bq_client.query(bq_query_str)

                for row in query_job:
                    csvwriter.writerow([actor, start.split('-')[0], end.split('-')[0], row[0], row[1], row[2]])

                print("Bytes (GB) actually processed: {}".format(query_job.total_bytes_processed/1024/1024/1024))

In [ ]:
import csv
import os

client = bigquery.Client()

for name, code in country_name_gdelt_code.items():
    if code in top_sources_fips.keys():
        get_summary_stats_bq(name, code, top_sources_fips[code], client)

In [ ]:
from google.cloud import bigquery

def get_top_sources_bq(asian_african_fips, bq_client):
    bq_query_str = '''
    SELECT 
        MentionSourceName, COUNT(MentionSourceName) as NumOccur
    FROM 
        gdelt-bq.gdeltv2.eventmentions_partitioned m 
    WHERE 
        m.GLOBALEVENTID in (SELECT 
                                    GLOBALEVENTID 
                            FROM 
                                    gdelt-bq.gdeltv2.events e 
                            WHERE 
                                    (e.Actor1CountryCode = "CHN" OR e.Actor2CountryCode = "CHN") AND
                                    e.ActionGeo_CountryCode in ('AF', 'AJ', 'BA', 'BG', 'AM', 'BT', 'IO', 'BX', 'BM', 'CB', 'CE', 'CH', 'TW', 'KT', 'CK', 'CY', 'GG', 'WE', 'HK', 'IN', 'ID', 'IR', 'IZ', 'IS', 'JA', 'KZ', 'JO', 'KN', 'KS', 'KU', 'KG', 'LA', 'LE', 'MC', 'MY', 'MV', 'MG', 'MU', 'NP', 'PK', 'RP', 'TT', 'QA', 'RS', 'SA', 'SN', 'VM', 'SY', 'TI', 'TH', 'AE', 'TU', 'TX', 'UZ', 'YM', 'AG', 'AO', 'BC', 'BY', 'CM', 'CV', 'CT', 'CD', 'CN', 'MF', 'CF', 'CG', 'BN', 'EK', 'ET', 'ER', 'DJ', 'GB', 'GA', 'GH', 'GV', 'IV', 'KE', 'LT', 'LI', 'LY', 'MA', 'MI', 'ML', 'MR', 'MP', 'MO', 'MZ', 'WA', 'NG', 'NI', 'PU', 'RE', 'RW', 'SH', 'TP', 'SG', 'SE', 'SL', 'SO', 'SF', 'ZI', 'OD', 'WI', 'SU', 'WZ', 'TO', 'TS', 'UG', 'EG', 'TZ', 'UV', 'ZA')
                            )
    GROUP BY MentionSourceName
    ORDER BY COUNT(MentionSourceName) DESC
    LIMIT 10000;
    '''

    with open("ChinaActorSourceCountsInAsiaAfrica.csv", 'w') as csvfile:
        csvwriter = csv.writer(csvfile) 
        csvwriter.writerow(fields) 

        query_job = bq_client.query(bq_query_str)

        for row in query_job:
            csvwriter.writerow([row[0], row[1]])

        print("Bytes (GB) actually processed: {}".format(query_job.total_bytes_processed/1024/1024/1024))

In [ ]:
print(tuple(country_name_gdelt_code.values()))

**Western and Chinese reporting** (i.e. popular western and Chinese news outlets) tone statistics for various actors (CHN, RUS, IND, USA, FRA, ALL) in the target country for the target time period.

In [38]:
from google.cloud import bigquery

def get_source_stats_bq(target_country, target_fips_code, bq_client):
    western_sources = ['reuters.com', 'yahoo.com', 'dailymail.co.uk', 'ap.org', 'msn.com', 'BBC Monitoring', 
                       'washingtonpost.com', 'nytimes.com', 'bloomberg.com']
    chinese_sources = ['chinadaily.com.cn', 'xinhuanet.com', 'china.org.cn', 'ecns.cn', 'scmp.com', 
                       'globaltimes.cn', 'en.people.cn', 'peopledaily.com.cn', 'sina.com', 'chinanationalnews.com', 
                       'people.com.cn', 'thenews.com.pk', 'shanghaisun.com', 'bjreview.com']
    for sources in ["western", "chinese"]:
        sources_str = '("' + '", "'.join(western_sources if sources == "western" else chinese_sources) + '")'

        for actor in ['CHN', 'RUS', 'IND', 'USA', 'FRA', 'ALL']: # CAMEO Country Code
            actor_where = '(e.Actor1CountryCode = "' + actor + '" OR e.Actor2CountryCode = "' + actor + '") AND' if actor != 'ALL' else ''
            for start, end in [("2015-02-18", "2015-12-31"),
                               ("2016-01-01", "2016-12-31"), 
                               ("2017-01-01", "2017-12-31"), 
                               ("2018-01-01", "2018-12-31"), 
                               ("2019-01-01", "2019-12-31"), 
                               ("2020-01-01", "2020-12-31"),
                               ("2021-01-01", "2021-03-08"),
                               ("2015-02-18", "2021-03-08")]:
                bq_query_str = '''
                SELECT 
                    avg(MentionDocTone) as AverageTone, stddev(MentionDocTone) as StdDevTone, count(MentionDocTone) as NumArticles
                FROM 
                    gdelt-bq.gdeltv2.eventmentions_partitioned m 
                WHERE 
                    m.GLOBALEVENTID in (SELECT 
                                              GLOBALEVENTID 
                                        FROM 
                                              gdelt-bq.gdeltv2.events_partitioned e 
                                        WHERE 
                                              ''' + actor_where + '''
                                                e.ActionGeo_CountryCode = "''' + target_fips_code + '''" AND
                                              e._PARTITIONTIME BETWEEN TIMESTAMP("''' + start + '''") AND 
                                              TIMESTAMP("''' + end + '''")) AND
                    m._PARTITIONTIME BETWEEN TIMESTAMP("''' + start + '''") AND TIMESTAMP("''' + end + '''") AND
                    m.MentionSourceName in ''' + sources_str + ''';
                '''

                print("\n\nSources: {}\nCountry: {}\nActor: {}\nStart: {}\nEnd: {}".format(sources, target_country, actor, start, end))
                fields = ['CountryCAMEOCode', 'StartYear', 'EndYear', 'Mean', 'StdDev', 'NumArticles']
                target_country_dir = 'CountrySourceCounts/' + target_country
                with open(target_country_dir + "/" + sources + "_stats_" + start + "_" + end + "_" + actor + ".csv", 'w') as csvfile:
                    csvwriter = csv.writer(csvfile) 
                    csvwriter.writerow(fields) 

                    query_job = bq_client.query(bq_query_str)

                    for row in query_job:
                        csvwriter.writerow([actor, start.split('-')[0], end.split('-')[0], row[0], row[1], row[2]])

                    print("Bytes (GB) actually processed: {}".format(query_job.total_bytes_processed/1024/1024/1024))

In [39]:
import csv
import os

client = bigquery.Client()

for name, code in country_name_gdelt_code.items():
    get_source_stats_bq(name, code, client)



Sources: western
Country: Afghanistan, Islamic Republic of
Actor: CHN
Start: 2015-02-18
End: 2015-12-31
Bytes (GB) actually processed: 9.680023287422955


Sources: western
Country: Afghanistan, Islamic Republic of
Actor: CHN
Start: 2016-01-01
End: 2016-12-31
Bytes (GB) actually processed: 12.328904586844146


Sources: western
Country: Afghanistan, Islamic Republic of
Actor: CHN
Start: 2017-01-01
End: 2017-12-31
Bytes (GB) actually processed: 11.27438600640744


Sources: western
Country: Afghanistan, Islamic Republic of
Actor: CHN
Start: 2018-01-01
End: 2018-12-31
Bytes (GB) actually processed: 10.34727443754673


Sources: western
Country: Afghanistan, Islamic Republic of
Actor: CHN
Start: 2019-01-01
End: 2019-12-31
Bytes (GB) actually processed: 8.59046896547079


Sources: western
Country: Afghanistan, Islamic Republic of
Actor: CHN
Start: 2020-01-01
End: 2020-12-31
Bytes (GB) actually processed: 6.426926559768617


Sources: western
Country: Afghanistan, Islamic Republic of
Actor: CH

Bytes (GB) actually processed: 6.42692730948329


Sources: chinese
Country: Afghanistan, Islamic Republic of
Actor: CHN
Start: 2021-01-01
End: 2021-03-08
Bytes (GB) actually processed: 1.0939229698851705


Sources: chinese
Country: Afghanistan, Islamic Republic of
Actor: CHN
Start: 2015-02-18
End: 2021-03-08
Bytes (GB) actually processed: 59.74190756306052


Sources: chinese
Country: Afghanistan, Islamic Republic of
Actor: RUS
Start: 2015-02-18
End: 2015-12-31
Bytes (GB) actually processed: 9.680023287422955


Sources: chinese
Country: Afghanistan, Islamic Republic of
Actor: RUS
Start: 2016-01-01
End: 2016-12-31
Bytes (GB) actually processed: 12.328904586844146


Sources: chinese
Country: Afghanistan, Islamic Republic of
Actor: RUS
Start: 2017-01-01
End: 2017-12-31
Bytes (GB) actually processed: 11.27438600640744


Sources: chinese
Country: Afghanistan, Islamic Republic of
Actor: RUS
Start: 2018-01-01
End: 2018-12-31
Bytes (GB) actually processed: 10.34727443754673


Sources: chinese
C

Bytes (GB) actually processed: 10.34727443754673


Sources: western
Country: Azerbaijan, Republic of
Actor: RUS
Start: 2019-01-01
End: 2019-12-31
Bytes (GB) actually processed: 8.59046896547079


Sources: western
Country: Azerbaijan, Republic of
Actor: RUS
Start: 2020-01-01
End: 2020-12-31
Bytes (GB) actually processed: 6.42692730948329


Sources: western
Country: Azerbaijan, Republic of
Actor: RUS
Start: 2021-01-01
End: 2021-03-08
Bytes (GB) actually processed: 1.0939229698851705


Sources: western
Country: Azerbaijan, Republic of
Actor: RUS
Start: 2015-02-18
End: 2021-03-08
Bytes (GB) actually processed: 59.74190756306052


Sources: western
Country: Azerbaijan, Republic of
Actor: IND
Start: 2015-02-18
End: 2015-12-31
Bytes (GB) actually processed: 9.680023287422955


Sources: western
Country: Azerbaijan, Republic of
Actor: IND
Start: 2016-01-01
End: 2016-12-31
Bytes (GB) actually processed: 12.328904586844146


Sources: western
Country: Azerbaijan, Republic of
Actor: IND
Start: 2017-

Bytes (GB) actually processed: 8.59046896547079


Sources: chinese
Country: Azerbaijan, Republic of
Actor: IND
Start: 2020-01-01
End: 2020-12-31
Bytes (GB) actually processed: 6.42692730948329


Sources: chinese
Country: Azerbaijan, Republic of
Actor: IND
Start: 2021-01-01
End: 2021-03-08
Bytes (GB) actually processed: 1.0939229698851705


Sources: chinese
Country: Azerbaijan, Republic of
Actor: IND
Start: 2015-02-18
End: 2021-03-08
Bytes (GB) actually processed: 59.74190756306052


Sources: chinese
Country: Azerbaijan, Republic of
Actor: USA
Start: 2015-02-18
End: 2015-12-31
Bytes (GB) actually processed: 9.680023287422955


Sources: chinese
Country: Azerbaijan, Republic of
Actor: USA
Start: 2016-01-01
End: 2016-12-31
Bytes (GB) actually processed: 12.328904586844146


Sources: chinese
Country: Azerbaijan, Republic of
Actor: USA
Start: 2017-01-01
End: 2017-12-31
Bytes (GB) actually processed: 11.27438600640744


Sources: chinese
Country: Azerbaijan, Republic of
Actor: USA
Start: 2018-

Bytes (GB) actually processed: 6.42692730948329


Sources: western
Country: Bahrain, Kingdom of
Actor: USA
Start: 2021-01-01
End: 2021-03-08
Bytes (GB) actually processed: 1.0939229698851705


Sources: western
Country: Bahrain, Kingdom of
Actor: USA
Start: 2015-02-18
End: 2021-03-08
Bytes (GB) actually processed: 59.74190756306052


Sources: western
Country: Bahrain, Kingdom of
Actor: FRA
Start: 2015-02-18
End: 2015-12-31
Bytes (GB) actually processed: 9.680023287422955


Sources: western
Country: Bahrain, Kingdom of
Actor: FRA
Start: 2016-01-01
End: 2016-12-31
Bytes (GB) actually processed: 12.328904586844146


Sources: western
Country: Bahrain, Kingdom of
Actor: FRA
Start: 2017-01-01
End: 2017-12-31
Bytes (GB) actually processed: 11.27438600640744


Sources: western
Country: Bahrain, Kingdom of
Actor: FRA
Start: 2018-01-01
End: 2018-12-31
Bytes (GB) actually processed: 10.34727443754673


Sources: western
Country: Bahrain, Kingdom of
Actor: FRA
Start: 2019-01-01
End: 2019-12-31
Bytes

...

Bytes (GB) actually processed: 8.59046896547079


Sources: western
Country: Tanzania, United Republic of
Actor: USA
Start: 2020-01-01
End: 2020-12-31
Bytes (GB) actually processed: 6.426955249160528


Sources: western
Country: Tanzania, United Republic of
Actor: USA
Start: 2021-01-01
End: 2021-03-08
Bytes (GB) actually processed: 1.0939341140910983


Sources: western
Country: Tanzania, United Republic of
Actor: USA
Start: 2015-02-18
End: 2021-03-08
Bytes (GB) actually processed: 59.74194664694369


Sources: western
Country: Tanzania, United Republic of
Actor: FRA
Start: 2015-02-18
End: 2015-12-31
Bytes (GB) actually processed: 9.680023287422955


Sources: western
Country: Tanzania, United Republic of
Actor: FRA
Start: 2016-01-01
End: 2016-12-31
Bytes (GB) actually processed: 12.328904586844146


Sources: western
Country: Tanzania, United Republic of
Actor: FRA
Start: 2017-01-01
End: 2017-12-31
Bytes (GB) actually processed: 11.27438600640744


Sources: western
Country: Tanzania, United

Bytes (GB) actually processed: 10.34727443754673


Sources: chinese
Country: Tanzania, United Republic of
Actor: FRA
Start: 2019-01-01
End: 2019-12-31
Bytes (GB) actually processed: 8.59046896547079


Sources: chinese
Country: Tanzania, United Republic of
Actor: FRA
Start: 2020-01-01
End: 2020-12-31
Bytes (GB) actually processed: 6.426955436356366


Sources: chinese
Country: Tanzania, United Republic of
Actor: FRA
Start: 2021-01-01
End: 2021-03-08
Bytes (GB) actually processed: 1.093934141099453


Sources: chinese
Country: Tanzania, United Republic of
Actor: FRA
Start: 2015-02-18
End: 2021-03-08
Bytes (GB) actually processed: 59.74194686114788


Sources: chinese
Country: Tanzania, United Republic of
Actor: ALL
Start: 2015-02-18
End: 2015-12-31
Bytes (GB) actually processed: 9.290046487934887


Sources: chinese
Country: Tanzania, United Republic of
Actor: ALL
Start: 2016-01-01
End: 2016-12-31
Bytes (GB) actually processed: 11.823256084695458


Sources: chinese
Country: Tanzania, United 

Bytes (GB) actually processed: 57.287472317926586


Sources: chinese
Country: Burkina Faso
Actor: CHN
Start: 2015-02-18
End: 2015-12-31
Bytes (GB) actually processed: 9.680023287422955


Sources: chinese
Country: Burkina Faso
Actor: CHN
Start: 2016-01-01
End: 2016-12-31
Bytes (GB) actually processed: 12.328904586844146


Sources: chinese
Country: Burkina Faso
Actor: CHN
Start: 2017-01-01
End: 2017-12-31
Bytes (GB) actually processed: 11.27438600640744


Sources: chinese
Country: Burkina Faso
Actor: CHN
Start: 2018-01-01
End: 2018-12-31
Bytes (GB) actually processed: 10.34727443754673


Sources: chinese
Country: Burkina Faso
Actor: CHN
Start: 2019-01-01
End: 2019-12-31
Bytes (GB) actually processed: 8.59046896547079


Sources: chinese
Country: Burkina Faso
Actor: CHN
Start: 2020-01-01
End: 2020-12-31
Bytes (GB) actually processed: 6.426955837756395


Sources: chinese
Country: Burkina Faso
Actor: CHN
Start: 2021-01-01
End: 2021-03-08
Bytes (GB) actually processed: 1.0939343655481935


So

Bytes (GB) actually processed: 10.34727443754673


Sources: western
Country: Zambia, Republic of
Actor: RUS
Start: 2019-01-01
End: 2019-12-31
Bytes (GB) actually processed: 8.59046896547079


Sources: western
Country: Zambia, Republic of
Actor: RUS
Start: 2020-01-01
End: 2020-12-31
Bytes (GB) actually processed: 6.426955837756395


Sources: western
Country: Zambia, Republic of
Actor: RUS
Start: 2021-01-01
End: 2021-03-08
Bytes (GB) actually processed: 1.0939343655481935


Sources: western
Country: Zambia, Republic of
Actor: RUS
Start: 2015-02-18
End: 2021-03-08
Bytes (GB) actually processed: 59.74194748699665


Sources: western
Country: Zambia, Republic of
Actor: IND
Start: 2015-02-18
End: 2015-12-31
Bytes (GB) actually processed: 9.680023287422955


Sources: western
Country: Zambia, Republic of
Actor: IND
Start: 2016-01-01
End: 2016-12-31
Bytes (GB) actually processed: 12.328904586844146


Sources: western
Country: Zambia, Republic of
Actor: IND
Start: 2017-01-01
End: 2017-12-31
Bytes

Bytes (GB) actually processed: 6.426955837756395


Sources: chinese
Country: Zambia, Republic of
Actor: IND
Start: 2021-01-01
End: 2021-03-08
Bytes (GB) actually processed: 1.0939343655481935


Sources: chinese
Country: Zambia, Republic of
Actor: IND
Start: 2015-02-18
End: 2021-03-08
Bytes (GB) actually processed: 59.74194748699665


Sources: chinese
Country: Zambia, Republic of
Actor: USA
Start: 2015-02-18
End: 2015-12-31
Bytes (GB) actually processed: 9.680023287422955


Sources: chinese
Country: Zambia, Republic of
Actor: USA
Start: 2016-01-01
End: 2016-12-31
Bytes (GB) actually processed: 12.328904586844146


Sources: chinese
Country: Zambia, Republic of
Actor: USA
Start: 2017-01-01
End: 2017-12-31
Bytes (GB) actually processed: 11.27438600640744


Sources: chinese
Country: Zambia, Republic of
Actor: USA
Start: 2018-01-01
End: 2018-12-31
Bytes (GB) actually processed: 10.34727443754673


Sources: chinese
Country: Zambia, Republic of
Actor: USA
Start: 2019-01-01
End: 2019-12-31
Byte

In [1]:
import pandas as pd

In [2]:
from glob import glob

In [14]:
local_paths = glob('CountrySourceCounts/*/local.csv')

,path
0,"CountrySourceCounts/Israel, State of/local.csv"
1,"CountrySourceCounts/Somalia, Somali Republic/l..."
2,"CountrySourceCounts/Iraq, Republic of/local.csv"
3,"CountrySourceCounts/Tajikistan, Republic of/lo..."
4,"CountrySourceCounts/Jordan, Hashemite Kingdom ..."
...,...
89,"CountrySourceCounts/Eritrea, State of/local.csv"
90,"CountrySourceCounts/Mauritania, Islamic Republ..."
91,"CountrySourceCounts/India, Republic of/local.csv"
92,"CountrySourceCounts/Egypt, Arab Republic of/lo..."


In [39]:
df = pd.read_csv(local_paths[0])
df['Country'] = local_paths[0].split('/')[-2]
for lp in local_paths[1:]:
    df2 = pd.read_csv(lp)
    df2['Country'] = lp.split('/')[-2]
    df = pd.concat([df,df2],ignore_index=True)[['Country', 'Source Name']]
groups = df.groupby('Country')['Source Name'].apply(lambda x: ', '.join(list(x)))
df3 = groups.reset_index(name = 'Local Sources')    
df3

,Country,Local Sources
0,"Afghanistan, Islamic Republic of","tolonews.com, pajhwok.com, outlookafghanistan...."
1,"Algeria, People's Democratic Republic of","djazairess.com, aps.dz, lemaghrebdz.com, elbil..."
2,"Angola, Republic of","allafrica.com, angop.ao, portalangop.co.ao, jo..."
3,"Armenia, Republic of","armenpress.am, news.am, panorama.am, panarmeni..."
4,"Azerbaijan, Republic of","azertag.az, azernews.az, en.trend.az, today.az..."
...,...,...
86,"Uzbekistan, Republic of","uza.uz, uzdaily.com, jahonnews.uz"
87,"Vietnam, Socialist Republic of","english.vietnamnet.vn, en.vietnamplus.vn, tuoi..."
88,Yemen,"yemenakhbar.com, barakish.net, alsahwa-yemen.net"
89,"Zambia, Republic of","lusakatimes.com, znbc.co.zm, daily-mail.co.zm,..."


In [42]:
df3.to_csv('country_local_sources.csv', sep='|', index=False)